## Импорты

In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import (
    LoraConfig,
    PeftModel,
    # get_peft_model,
)

from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format, DataCollatorForCompletionOnlyLM
from dataclasses import dataclass
import numpy as np
import pandas as pd
import traceback
import torch
from GPUtil import showUtilization as gpu_usage
from peft import get_peft_model
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MaxNLocator
from transformers import TrainerCallback
import transformers
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MaxNLocator


## Конфигурация, переменные и сервисные функции

In [2]:
@dataclass
class Config:
    model_name = "unsloth/Llama-3.2-1B-Instruct"
    new_model = "llama-3.1-8b-chat-house"
    torch_dtype = torch.float16
    attn_implementation = "eager"
cfg = Config()

# Очистка GPU
def clear_gpu_memory():
    print("\nInitial GPU Usage")
    gpu_usage()  # Показывает текущее использование GPU

    torch.cuda.empty_cache()  # Очищает кеш CUDA

    print("\nGPU Usage after emptying the cache")
    gpu_usage()  # Показывает использование GPU после очистки кеша

## Подготовка модели

In [ ]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=cfg.torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Загрузка модели и токенизатора
model = AutoModelForCausalLM.from_pretrained(
    cfg.model_name,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=cfg.attn_implementation
)
tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)

# Явный сброс существующего чат-шаблона перед настройкой
if tokenizer.chat_template is not None:
    tokenizer.chat_template = None

# 1. Настройка формата чата (теперь можно безопасно добавлять)
model, tokenizer = setup_chat_format(model, tokenizer)

# 2. Добавляем pad_token после настройки чата
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<|pad|>'})

# 3. Синхронизация размеров ПОСЛЕ ВСЕХ изменений
model.resize_token_embeddings(len(tokenizer))

# Дополнительные настройки
tokenizer.padding_side = 'right'  # Для корректной работы с пакетами

# Проверка размеров
print(f"[Проверка] Размер словаря: {len(tokenizer)}")
print(f"[Проверка] Размер эмбеддингов: {model.get_input_embeddings().weight.shape[0]}")

# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

# Применяем LoRA
model = get_peft_model(model, peft_config)

# Загрузка данных
dataset_dict = load_dataset("json", data_files="../data/processed/context_answer.json")
dataset = dataset_dict["train"] if "train" in dataset_dict else dataset_dict

# Форматирование данных
def format_chat_template(row):
    row_json = [{"role": "user", "content": row["q"]},
               {"role": "assistant", "content": row["a"]}]
    text = tokenizer.apply_chat_template(row_json, tokenize=False)
    return text

# Создание data collator
response_template = "assistant\n"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# Разделение данных
dataset_sh = dataset.shuffle(seed=2024).select(range(len(dataset)))
dataset_sh = dataset_sh.train_test_split(0.1)

# Настройка аргументов обучения
training_arguments = TrainingArguments(
    output_dir=cfg.new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",
    eval_steps=50,
    logging_steps=100,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    group_by_length=True,
    logging_dir='../logs',
    report_to=["mlflow"],
    run_name="Llama-3.2-house",
)

# Создание тренера
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_sh["train"],
    eval_dataset=dataset_sh["test"],
    peft_config=peft_config,
    tokenizer=tokenizer,
    formatting_func=format_chat_template,
    args=training_arguments,
    data_collator=collator,
)

# 1. Сбор метрик во время обучения
class MetricTracker(transformers.TrainerCallback):
    def __init__(self):
        self.metrics = {'train': [], 'eval': []}
        self.current_step = 0
        
    def on_log(self, args, state, control, logs=None, **kwargs):
        if 'loss' in logs:
            self.metrics['train'].append((self.current_step, logs['loss']))
        if 'eval_loss' in logs:
            self.metrics['eval'].append((self.current_step, logs['eval_loss']))
        self.current_step = state.global_step
        

## Процесс обучения

In [ ]:
# Очистка памяти GPU перед обучением
clear_gpu_memory()

# Добавляем колбэк перед обучением
metric_tracker = MetricTracker()
trainer.add_callback(metric_tracker)

# Обучение
try:
    trainer.train()
except Exception as e:
    print("Произошла ошибка:")
    traceback.print_exc()
    print(f"Тип ошибки: {type(e).__name__}")
    print(f"Сообщение ошибки: {str(e)}")

model.resize_token_embeddings(len(tokenizer))


## Визуализация результатов

In [ ]:
# 2. Визуализация результатов
plt.figure(figsize=(16, 8))

# Конфигурация стиля
colors = {'train': '#3498db', 'eval': '#e74c3c'}
alpha = 0.2
smooth_window = 50

# Функция для сглаживания
def smooth_data(steps, values, window):
    df = pd.DataFrame({'step': steps, 'value': values})
    return df['value'].rolling(window, min_periods=1).mean().values

# Построение графиков
for metric_type in ['train', 'eval']:
    if len(metric_tracker.metrics[metric_type]) > 0:
        steps, values = zip(*metric_tracker.metrics[metric_type])
        steps, values = np.array(steps), np.array(values)
        
        # Сырые данные
        plt.scatter(steps, values, alpha=alpha, 
                   color=colors[metric_type], s=15, 
                   label=f'{metric_type.capitalize()} Loss (Raw)')
        
        # Сглаженные данные
        if len(values) > smooth_window:
            smoothed = smooth_data(steps, values, smooth_window)
            plt.plot(steps, smoothed, 
                    color=colors[metric_type], 
                    linewidth=2,
                    label=f'{metric_type.capitalize()} Loss (Smoothed)')

# Разметка эпох и оценок
max_step = max([step for step, _ in metric_tracker.metrics['train']])
eval_steps = [step for step, _ in metric_tracker.metrics['eval']]

# Вертикальные линии для оценок
for step in eval_steps:
    plt.axvline(step, color='grey', linestyle='--', alpha=0.4, linewidth=0.8)

# Аннотации
plt.title('QLoRA Training Dynamics', fontsize=14, pad=20)
plt.xlabel('Training Steps', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend(loc='upper right')

# Настройка осей
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
plt.grid(True, alpha=0.2)
plt.tight_layout()

# 3. Дополнительная панель с ключевыми метриками
ax2 = plt.gca().inset_axes([0.72, 0.68, 0.25, 0.25])
metrics_text = [
    f"Final Train Loss: {metric_tracker.metrics['train'][-1][1]:.2f}",
    f"Best Eval Loss: {min([v for _, v in metric_tracker.metrics['eval']]):.2f}",
    f"Total Steps: {max_step}",
    f"Eval Frequency: {training_arguments.eval_steps} steps"
]

ax2.text(0.1, 0.8, "\n".join(metrics_text), fontsize=9)
ax2.axis('off')

plt.show()


## Сохранение модели для дальнейшего использования и тестирования

Сохраняем отдельно:
- Токенизатор с новыми токенами
- Адаптеры LoRA
- Конфигурацию модели

In [5]:
path_to_save = "../models/Llama-finetuned"

# Убедимся, что размеры синхронизированы перед сохранением
model.resize_token_embeddings(len(tokenizer))

# Сохраняем токенизатор
tokenizer.save_pretrained(path_to_save)

# Сохраняем адаптеры LoRA отдельно
model.save_pretrained(path_to_save) 


def load_custom_model(model_path, base_model_name):
    """Загрузка базовой модели"""
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=bnb_config,
        device_map="auto"
    )
    
    # Синхронизируем с токенизатором
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    base_model.resize_token_embeddings(len(tokenizer))
    
    # Загружаем адаптеры
    model = PeftModel.from_pretrained(base_model, model_path)
    
    return model, tokenizer

# Загрузка модели
loaded_model, loaded_tokenizer = load_custom_model(path_to_save, cfg.model_name)

# Проверка размеров
print(f"Tokenizer size: {len(loaded_tokenizer)}")
print(f"Model embeddings: {loaded_model.get_input_embeddings().weight.shape[0]}")

# Объединение для инференса
merged_model = loaded_model.merge_and_unload()
merged_model.save_pretrained("../models/Llama-merged")

/opt/conda/lib/python3.11/site-packages/peft/utils/save_and_load.py:241: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Tokenizer size: 128258
Model embeddings: 128258


/opt/conda/lib/python3.11/site-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
